<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/manual_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GAN-Functions
Created on Wed Nov 18 07:56:39 2020

@author: Wolfgang Reuter

Inspired and altered from:
    https://github.com/platonovsimeon/dcgan-facegenerator

In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Tensor Flow Version: 2.2.0
Keras Version: 2.3.0-tf

Python 3.7.9 (default, Aug 31 2020, 12:42:55) 
[GCC 7.3.0]
Pandas 1.1.3
Scikit-Learn 0.23.2
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


# Imports

In [7]:
# =============================================================================
# Imports
# =============================================================================

# Model functions from keras
import tensorflow as tf


from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten, \
                         BatchNormalization, Activation, ZeroPadding2D


#from tensorflow.keras.layers.advanced_activations import LeakyReLU  ---geht nicht
from tensorflow.keras.layers import LeakyReLU


#from tensorflow.keras.layers.convolutional import UpSampling2D, Conv2D ---geht nicht
from tensorflow.keras.layers import UpSampling2D, Conv2D


from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam

# matplotlib will help with displaying the results
import matplotlib.pyplot as plt
# numpy for some mathematical operations
import numpy as np
# PIL for opening,resizing and saving images
from PIL import Image
# tqdm for a progress bar when loading the dataset
from tqdm import tqdm

#os library is needed for extracting filenames from the dataset folder.
import os

# Custom functions
# in ipynb see functions
import utils.gan_functions


ModuleNotFoundError: No module named 'utils'

# Paths and Variables

In [11]:
# =============================================================================
# Paths and Variables
# =============================================================================

# Data Parameters
imgs_dir = 'data'

image_height, image_width, channels = 64, 64, 3
image_shape = (image_width, image_height, channels)

random_noise_dimension = 100
rows, columns = 4, 8

save_images_interval = 100

target_dir = 'generated_faces'

target_fn = "/generated"

# Model parameters

use_pretrained_model = False

pretrained_model_path_generator = "saved_models/face_generator.h5"
pretrained_model_path_discriminator = "saved_models/face_discriminator.h5"

epochs = 5000
batch_size = 32

start_epoch = 0
if use_pretrained_model:
    assert(start_epoch == 0)

optimizer = Adam(0.0002, 0.5)
optimizer = Adam(0.0002, 0.5)

# TODO: Rewrite get_random_noise() to use batch_size
assert(rows * columns == batch_size)

# Functions

In [ ]:
# =============================================================================
# Functions
# =============================================================================

# A compendium of available functions in gan_functions 

# get_random_noise(rows, columns, random_noise_dimensions)

# build_generator(random_noise_dimension, channels)

# build_discriminator()

In [13]:
#==============================================================================
# Functions (from gan_functions)
# =============================================================================

def get_random_noise(rows, columns, random_noise_dimension):
    """

    Parameters
    ----------
    rows : Integer
        Pixel size of the height of a "noise image"
    columns : Integer
        Pixel size of the width of an "noise image"
    random_noise_dimension : Integer
        Number of channels of a "noise image"

    Returns
    -------
    2-dim numpy array
        Array of shape (rows * columns, random_noise_dimensions) with 
        normally distributed values with mean 0 and standard deviation 1

    """
    return np.random.normal(0, 1, (rows * columns, random_noise_dimension))

In [14]:
def get_training_data(datafolder, image_width, image_height, channels):
    """
    Loads the training data from a specified folder
    
    Parameters
    ----------
    datafolder : String
        Path to directory where the training images are stored
    image_width : Integer
        Pixel width of the images (Nr. of columns)
    image_height : Integer
        Pixel height of the images (Nr. of rows)
    channels : Integer
        Number of color channels

    Returns
    -------
    training_data : 4-dim numpy array
        The training data as numpy array of shaape:    
            (Nr. images, image_height, image_width, Nr. channels) 

    """
    print("Loading training data...")

    training_data = []
    #Finds all files in datafolder
    filenames = os.listdir(datafolder)
    for filename in tqdm(filenames):
        #Combines folder name and file name.
        path = os.path.join(datafolder,filename)
        #Opens an image as an Image object.
        image = Image.open(path)
        #Resizes to a desired size.
        image = image.resize((image_width,image_height),Image.ANTIALIAS)
        #Creates an array of pixel values from the image.
        pixel_array = np.asarray(image)
        
        # Clip alpha channel, if existant
        if pixel_array.shape[2] > channels: 
            pixel_array = pixel_array[:,:,:channels]

        training_data.append(pixel_array)

    #training_data is converted to a numpy array
    training_data = \
        np.reshape(training_data,(-1, image_width, image_height, channels))
    return training_data

In [12]:
def build_generator(random_noise_dimension, channels):
    ''' 
    create generator-model
    
    Parameter:
        random_noise_dimension : Number of channels of a "noise image" (Integer)
        channels : Number of color channels (Integer)
        
    Output:
        Generator-Model
    '''
    #Generator attempts to fool discriminator by generating new images.
    model = Sequential()
    model.add(Dense(256*4*4,activation="relu",\
                    input_dim=random_noise_dimension))
    model.add(Reshape((4,4,256)))
    

    # Four layers of upsampling, convolution, batch normalization 
    # and activation.
    # 1. Upsampling: Input data is repeated. Default is (2,2). 
    #    In that case a 4x4x256 array becomes an 8x8x256 array.
    # 2. Convolution: If you are not familiar, you should watch 
    #    this video: https://www.youtube.com/watch?v=FTr3n7uBIuE
    # 3. Normalization normalizes outputs from convolution.
    # 4. Relu activation:  f(x) = max(0,x). If x < 0, then f(x) = 0.


    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))


    # Last convolutional layer outputs as many featuremaps as channels 
    # in the final image.
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    # model.add(Conv2D(channels, kernel_size=1, padding="same"))
    # tanh maps everything to a range between -1 and 1.
    model.add(Activation("tanh"))

    # show the summary of the model architecture
    model.summary()

    # Placeholder for the random noise input
    input = Input(shape=(random_noise_dimension,))
    #Model output
    generated_image = model(input)

    # Change the model type from Sequential to Model (functional API) 
    # More at: https://keras.io/models/model/.
    return Model(input,generated_image)


In [15]:
def build_discriminator(image_shape):
    #Discriminator attempts to classify real and generated images
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, \
                     input_shape=image_shape, padding="same"))
    #Leaky relu is similar to usual relu. If x < 0 then f(x) = x * alpha, 
    # otherwise f(x) = x.
    model.add(LeakyReLU(alpha=0.2))

    # Dropout blocks some connections randomly. This help the model 
    # to generalize better. 0.25 means that every connection has a 25% 
    # chance of being blocked.
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    # Zero padding adds additional rows and columns to the image. 
    # Those rows and columns are made of zeros.
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    # Flatten layer flattens the output of the previous layer to a single 
    # dimension.
    model.add(Flatten())
    # Outputs a value between 0 and 1 that predicts whether image is 
    # real or generated. 0 = generated, 1 = real.
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    input_image = Input(image_shape)

    #Model output given an image.
    validity = model(input_image)

    return Model(input_image, validity)

In [ ]:
def save_images(epoch, random_noise_dimension, generator, target_dir, 
                target_fn, start_epoch = 0):
    #Save generated images for demonstration purposes using matplotlib.pyplot.
    rows, columns = 5, 5
    noise = np.random.normal(0, 1, (rows * columns, random_noise_dimension))
    generated_images = generator.predict(noise)

    generated_images = 0.5 * generated_images + 0.5

    figure, axis = plt.subplots(rows, columns)
    image_count = 0
    for row in range(rows):
        for column in range(columns):
            axis[row,column].imshow(generated_images[image_count, :], \
                                    cmap='spring')
            axis[row,column].axis('off')
            image_count += 1
    figure.savefig(target_dir + target_fn + "_%d.png" % (start_epoch + epoch))
    plt.close()

# Get the training data and normalize it

In [18]:
# =============================================================================
# Get the training data and normalize it
# =============================================================================

#Get the real images
'''training_data = gan_functions.get_training_data(imgs_dir, 
                                                image_width,
                                                image_height, 
                                                channels)
only py
'''


'''I HAT IT---- where is the data, which path
I can first see that something works before I change ...  '''
training_data = get_training_data(imgs_dir,
                                  image_width,
                                  image_height,
                                  channels)
 
#Map all values to a range between -1 and 1.
training_data = training_data / 127.5 - 1.

Loading training data...


FileNotFoundError: [Errno 2] No such file or directory: 'data'

# Set up the labels for generated and real images



In [ ]:
# =============================================================================
# Set up the labels for generated and real images
# =============================================================================

# Two arrays of labels. Labels for real images: [1,1,1 ... 1,1,1], 
# labels for generated images: [0,0,0 ... 0,0,0]
labels_for_real_images = np.ones((batch_size,1)) - 0.15
labels_for_generated_images = np.zeros((batch_size,1))


# Set up generator and discriminator

In [ ]:
# =============================================================================
# Set up generator and discriminator
# =============================================================================

if use_pretrained_model:
    generator = load_model(pretrained_model_path_generator)
    discriminator = load_model(pretrained_model_path_discriminator)
else:
    generator = gan_functions.build_generator(random_noise_dimension, channels)
    discriminator = gan_functions.build_discriminator(image_shape)
    
discriminator.compile(loss="binary_crossentropy", 
                      optimizer=optimizer,
                      metrics=["accuracy"])

# Set up the actual GAN (= combined_model)
random_input = Input(shape=(random_noise_dimension,))
generated_image = generator(random_input)
discriminator.trainable = False
validity = discriminator(generated_image)#
combined_model = Model(random_input, validity) # This is the actual GAN

combined_model.compile(loss="binary_crossentropy",
                       optimizer=optimizer)

In [ ]:
# =============================================================================
# Train GAN
# =============================================================================

for epoch in range(epochs):
    # Select a random batch of real images
    indices = np.random.randint(0,training_data.shape[0],batch_size)
    real_images = training_data[indices]
    
    # Generate random noise for a whole batch.
    random_noise = gan_functions.get_random_noise(rows, 
                                                  columns, 
                                                  random_noise_dimension)
    
    discriminator.trainable = True
    
    #Generate a batch of new images.
    generated_images = generator.predict(random_noise)
    
    #Train the discriminator on real images.
    discriminator_loss_real = \
        discriminator.train_on_batch(real_images,
                                     labels_for_real_images)
    #Train the discriminator on generated images.
    discriminator_loss_generated = \
        discriminator.train_on_batch(generated_images,
                                     labels_for_generated_images)
    #Calculate the average discriminator loss.
    discriminator_loss = 0.5 * np.add(discriminator_loss_real,
                                      discriminator_loss_generated)
    
    # Train the generator using the combined model. Generator tries to trick 
    # discriminator into mistaking generated images as real.
    discriminator.trainable = False
    
    labels_for_tricking_discriminator = np.ones((batch_size,1))
    generator_loss = \
        combined_model.train_on_batch(random_noise,labels_for_tricking_discriminator)
    
    # Training ends above (one iteration) 
    # This is only for display and saving models
    print ("%d [Discriminator loss: %f, acc.: %.2f%%] [Generator loss: %f]" % \
           (epoch, discriminator_loss[0], 100*discriminator_loss[1], 
            generator_loss))

    if epoch % save_images_interval == 0:
        gan_functions.save_images(epoch, random_noise_dimension, generator, 
                                  target_dir, target_fn, start_epoch)
        
    #Save the model for a later use
    generator.save(pretrained_model_path_generator)
    discriminator.save(pretrained_model_path_discriminator)

